In [13]:
import numpy as np
import nltk
import random
import json

import torch


from model import NeuralNet
from utilities import bag_of_words, tokenize,stem
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()


In [4]:
with open('talks.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))



ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)


40 patterns
8 tags: ['assistance', 'delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
79 unique stemmed words: ["'s", 'a', 'accept', 'all', 'am', 'anyon', 'are', 'avaiabl', 'bye', 'can', 'card', 'care', 'cash', 'confus', 'credit', 'custom', 'day', 'deliveri', 'do', 'doe', 'for', 'funni', 'get', 'good', 'goodby', 'have', 'hear', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'it', 'item', 'joke', 'kind', 'know', 'later', 'laugh', 'list', 'long', 'lot', 'mastercard', 'me', 'menu', 'mode', 'my', 'need', 'of', 'ok', 'onli', 'order', 'pay', 'payment', 'paypal', 'pleas', 'problem', 'see', 'sell', 'ship', 'someth', 'take', 'talk', 'tell', 'thank', 'that', 'there', 'to', 'wait', 'want', 'what', 'when', 'where', 'which', 'will', 'with', 'you']


In [5]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)


79 8


In [11]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


In [14]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [15]:

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 0.5514
Epoch [200/1000], Loss: 0.0369
Epoch [300/1000], Loss: 0.0043
Epoch [400/1000], Loss: 0.0050
Epoch [500/1000], Loss: 0.0020
Epoch [600/1000], Loss: 0.0018
Epoch [700/1000], Loss: 0.0008
Epoch [800/1000], Loss: 0.0008
Epoch [900/1000], Loss: 0.0004
Epoch [1000/1000], Loss: 0.0002
final loss: 0.0002
training complete. file saved to data.pth


In [17]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('talks.json', 'r') as json_data:
    talks = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()



NeuralNet(
  (l1): Linear(in_features=79, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=8, bias=True)
  (relu): ReLU()
)

In [ ]:
bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand... I will connect with our Customer Care executive")

Let's chat! (type 'quit' to exit)
You: hey
Sam: Hi there, how can I help?
You: what do you sell
Sam: We only Sale tea and Coffee
You: price of tea
Sam: We sell coffee and tea
You: what in your menu
Sam: We sell coffee and tea
You: meantime tell me joke
Sam: Why did the hipster burn his mouth? He drank the coffee before it was cool.
You: joke time
Sam: Why did the hipster burn his mouth? He drank the coffee before it was cool.
